# Outline

- Dataset
 

- Metric


### Mount Folder

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Loading & Checking Example Data Based on Numpy & Pandas


In [2]:
# Setting up the required libraries
import numpy  as np
import pandas as pd

In [3]:
# The function for loading data
def load_LETOR4(file, num_features=46):
	'''
	:param file: the input file
	:param num_features: the number of features
	:return: the list of tuples, each tuple consists of qid, doc_reprs, doc_labels
	'''
  
	feature_cols = [str(f_index) for f_index in range(1, num_features + 1)]

	df = pd.read_csv(file, sep=" ", header=None)
	df.drop(columns=df.columns[[-2, -3, -5, -6, -8, -9]], axis=1, inplace=True)  # remove redundant keys
	assert num_features == len(df.columns) - 5

	for c in range(1, num_features +2):           							 # remove keys per column from key:value
		df.iloc[:, c] = df.iloc[:, c].apply(lambda x: x.split(":")[1])

	df.columns = ['rele_truth', 'qid'] + feature_cols + ['#docid', 'inc', 'prob']

	for c in ['rele_truth'] + feature_cols:
		df[c] = df[c].astype(np.float32)

	df['rele_binary'] = (df['rele_truth'] > 0).astype(np.float32)  # additional binarized column for later filtering

	list_Qs = []
	qids = df.qid.unique()
	np.random.shuffle(qids)
	for qid in qids:
		sorted_qdf = df[df.qid == qid].sort_values('rele_truth', ascending=False)

		doc_reprs = sorted_qdf[feature_cols].values
		doc_labels = sorted_qdf['rele_truth'].values

		list_Qs.append((qid, doc_reprs, doc_labels))

	#if buffer: pickle_save(list_Qs, file=perquery_file)

	return list_Qs

#### Check the data

In [4]:
file = '/content/drive/My Drive/Teaching/2020/KLIS-MLIR-2020/Data/vali_as_train.txt'

list_Qs = load_LETOR4(file=file)
print('The total number of queries:', len(list_Qs))

for (qid, doc_reprs, doc_labels) in list_Qs:
  print('qid:{}\t{}\t{}'.format(qid, doc_reprs.shape, doc_labels.shape))
  print('doc_reprs\n', doc_reprs)
  print('doc_labels\n', doc_labels)
  break

The total number of queries: 339
qid:7155	(40, 46)	(40,)
doc_reprs
 [[0.293605 0.       0.       ... 1.       0.272727 0.      ]
 [0.020349 0.       0.       ... 0.4      0.318182 0.      ]
 [0.438953 0.       0.       ... 0.6      0.136364 0.      ]
 ...
 [0.334302 0.       0.       ... 0.8      0.242424 0.      ]
 [0.008721 0.       0.       ... 0.4      0.318182 0.      ]
 [0.017442 0.       1.       ... 0.       0.060606 0.      ]]
doc_labels
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


### Loading & Checking Example Data Based on PTRanking


In [5]:
!pip install ptranking

     |████████████████████████████████| 122kB 6.4MB/s 


In [14]:
import torch
from ptranking.data.data_utils import LTRDataset, SPLIT_TYPE

data_id = 'MQ2007_Super'
file_train = '/content/drive/My Drive/Teaching/2020/KLIS-MLIR-2020/Data/vali_as_train.txt'

train_dataset = LTRDataset(SPLIT_TYPE.Train, data_id=data_id, file=file_train, buffer=False)

num_train_queries = train_dataset.__len__()

print('Dataset:\t', data_id)
print('Total train queries:\t', num_train_queries)

for qid, torch_batch_rankings, torch_batch_std_labels in train_dataset:
    print('torch_batch_std_labels', torch_batch_std_labels)
    
    doc_num = torch_batch_std_labels.size(1)

    print('torch_batch_rankings', torch_batch_rankings)
    break
    

Dataset:	 MQ2007_Super
Total train queries:	 339
torch_batch_std_labels tensor([[1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 2., 2.,
         2., 2., 2., 0., 1., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1.,
         0., 1., 1., 1.]])
torch_batch_rankings tensor([[[0.0333, 1.0000, 0.3333,  ..., 0.1250, 0.0000, 0.0000],
         [0.1059, 0.1000, 0.3333,  ..., 0.1250, 0.0108, 0.0000],
         [0.0231, 0.2000, 0.3333,  ..., 0.1250, 0.0215, 0.0000],
         ...,
         [0.1409, 0.0000, 0.0000,  ..., 0.2500, 0.1935, 0.0000],
         [1.0000, 0.0000, 0.0000,  ..., 0.1250, 0.0215, 0.0000],
         [0.2878, 0.0000, 0.0000,  ..., 0.3750, 0.0430, 0.0000]]])


## nDCG

nDCG is a widely used metric for evaluating the performance of a ranking model. The assumptions under nDCG are that:
- Highly relevant documents are more useful than marginally relevant document
- The lower the ranked position of a relevant document, the less useful it is for the user, since it is less likely to be examined

nDCG consists of different components, and is defined as follows:
The discounted cumulative gain (DCG) of a ranked list is given as:
$$DCG=\sum_{i=1}^{m}\frac{2^{y_{i}^{*}}-1}{\log(i+1)}$$
where $2^{y_{i}^{*}}-1$ is usually referred to as the gain value of the $i$-th document. 

We denote the maximum DCG value attained by the ideal ranking as $DCG^{*}$, then normalizing DCG with $DCG^{*}$ gives nDCG as follows:
$$nDCG=\frac{DCG}{DCG^{*}}=\frac{1}{DCG^{*}}\sum_{i=1}^{m}\frac{2^{y_{i}^{*}}-1}{\log(i+1)}$$

### Example evaluation
We denote the standard labels are $0$, $1$ and $2$. In particular, $0$ means non-relevant, $1$ means marginally relevant, $2$ means very relevant. In other words, the label reveals the degree how a document is relevant to the query.

Let's assume that there are five documents $D_1$, $D_2$, $D_3$, $D_4$ and $D_5$. The labels are given as: $D_1: 0$, $D_2: 2$, $D_3: 1$, $D_4: 0$ and $D_5: 1$.

Furthermore, we assume that there are two candidate functions (i.e., $f_1$ and $f_2$) to be compared. The relevance predictions by $f_1$ are: $D_1: 0.3$, $D_2: 0.4$, $D_3: 0.2$, $D_4: 0.5$ and $D_5: 1.1$. The relevance predictions by $f_2$ are: $D_1: 0.1$, $D_2: 1.5$, $D_3: 0.2$, $D_4: 0.4$ and $D_5: 0.6$. 

The nDCG scores for $f_1$ and $f_2$ are computed as follows:

First, we compute $DCG^{*}$ as:
$$DCG^{*}=(\frac{2^{2}-1}{\log_{2}(1+1)}+\frac{2^{1}-1}{\log_{2}(2+1)}+\frac{2^{1}-1}{\log_{2}(3+1)}+\frac{2^{0}-1}{\log_{2}(4+1)}+\frac{2^{0}-1}{\log_{2}(5+1)})=4.1309$$

Second, we sort the documents according to the predictions by $f_1$ and $f_2$, respectively. For $f_1$, we get the predicted ranking as: $D_5$, $D_4$, $D_2$, $D_1$ and $D_3$. The corresponding labels are: $[1, 0, 2, 0, 1]$.

For $f_2$, we get the predicted ranking as: $D_2$, $D_5$, $D_4$, $D_3$ and $D_1$. The corresponding labels are: $[2, 1, 0, 1, 0]$.

Third, for $f_1$, its nDCG score is computed as:
$$
DCG_{f-1}=(\frac{2^{1}-1}{\log_{2}(1+1)}+\frac{2^{0}-1}{\log_{2}(2+1)}+\frac{2^{2}-1}{\log_{2}(3+1)}+\frac{2^{0}-1}{\log_{2}(4+1)}+\frac{2^{1}-1}{\log_{2}(5+1)})=2.8868$$

Thire, for $f_2$, its nDCG score is computed as:
$$
DCG_{f-2}=(\frac{2^{2}-1}{\log_{2}(1+1)}+\frac{2^{1}-1}{\log_{2}(2+1)}+\frac{2^{0}-1}{\log_{2}(3+1)}+\frac{2^{1}-1}{\log_{2}(4+1)}+\frac{2^{0}-1}{\log_{2}(5+1)})=4.0616$$


### Example program for computing nDCG Based on Numpy

In [ ]:
def ndcg_at_k(sys_sorted_labels, ideal_sorted_labels, k):
	sys_discounted_cumu_gain_at_k = discounted_cumu_gain_at_k(sys_sorted_labels, cutoff=k)
	ideal_discounted_cumu_gain_at_k = discounted_cumu_gain_at_k(ideal_sorted_labels, cutoff=k)
	ndcg_at_k = sys_discounted_cumu_gain_at_k / ideal_discounted_cumu_gain_at_k
	return ndcg_at_k

In [ ]:
def discounted_cumu_gain_at_k(sorted_labels, cutoff):
	'''
	:param sorted_labels: ranked labels (either standard or predicted by a system) in the form of np array
	:param max_cutoff: the maximum rank position to be considered
	:param multi_lavel_rele: either the case of multi-level relevance or the case of listwise int-value, e.g., MQ2007-list
	:return: cumulative gains for each rank position
	'''
	nums = np.power(2.0, sorted_labels[0:cutoff]) - 1.0

	denoms = np.log2(np.arange(cutoff) + 2.0)  # discounting factor
	dited_cumu_gain = np.sum(nums / denoms)

	return dited_cumu_gain

Based on the above example, we compute the nDCG score for $f_1$ again.

The ideal ranking list is: $[2, 1, 1, 0, 0]$, The predicted ranking by $f_1$ is: $[1, 0, 2, 0, 1]$, thus we have

In [ ]:
sys_sorted_labels = [1, 0, 2, 0, 1] # the ranking predicted by a ranking model
ideal_sorted_labels=[2, 1, 1, 0, 0] # the ranking obtained according to the standard labels
nDCG = ndcg_at_k(sys_sorted_labels=sys_sorted_labels, ideal_sorted_labels=ideal_sorted_labels, k=5)
print(nDCG)

0.6988385132278441


### Example program for computing nDCG Based on PTRanking

In [19]:
from ptranking.metric.adhoc_metric import torch_nDCG_at_k, torch_nDCG_at_ks

sys_sorted_labels = torch.Tensor([1, 0, 2, 0, 1])# the ranking predicted by a ranking model
ideal_sorted_labels = torch.Tensor([2, 1, 1, 0, 0])# the ranking obtained according to the standard labels

ndcg_at_k = torch_nDCG_at_k(sys_sorted_labels.view(1, -1), ideal_sorted_labels.view(1, -1), k=5)
print(ndcg_at_k.size(), ndcg_at_k)

print()

ndcg_at_ks = torch_nDCG_at_ks(sys_sorted_labels.view(1, -1), ideal_sorted_labels.view(1, -1), ks=[1, 3, 5])
print(ndcg_at_ks.size(), ndcg_at_ks)

torch.Size([1, 1]) tensor([[0.6988]])

torch.Size([1, 3]) tensor([[0.3333, 0.6052, 0.6988]])
